In [1]:
import scipy.stats as st
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
def E(z):
    return np.mean(z)

def discard(a, n):
    return ('{0:.' + str(n) + 'f}').format(a).rstrip('0').rstrip('.')

np.random.seed(0)

In [3]:
N = 100
N_t= [100, 1000]

norm = st.norm(loc=0, scale=1)
x = norm.rvs(N)

m = E(x)
sigma = x.std()
print('m = ' + discard(m, 4) + '\nsigma =  ' + discard(sigma, 4) + '\n')

for N_test in N_t:
    table = pd.DataFrame([], columns = ['delta', 'n', 'p_i', 'temp_v'])
    table.columns.name = 'i'
    
    k = int(1.72 * np.cbrt(N_test))
    rs = norm.rvs(N_test)

    if k % 2 == 1:
        k -= 1

    left = -1
    right = 1
    step = (right - left) / (k - 2)

    delta = [left + i * step for i in range(0, k - 1)]

    n = np.zeros(k)
    for r in rs:
        last = True
        for i in range(0, len(delta)):
            if r < delta[i]:
                n[i] += 1
                last = False
                break
        if last:
            n[k - 1] += 1

    p = []
    model = st.norm(loc=m, scale=sigma)

    for i in range(0, k - 1):
        if i == 0:
            p.append(model.cdf(delta[0]))
        else:
            p.append(model.cdf(delta[i]) - model.cdf(delta[i - 1]))

    p.append(1 - model.cdf(delta[k - 2]))
    p = np.array(p)

    hi_square = 0
    for i in range(k):
        arr = []
        
        if i == k - 1:
            arr.append('inf')
        else:
            arr.append(delta[i])
            
        temp_v = ((n[i] - N_test * p[i]) ** 2) / (N_test * p[i])
        
        arr.append(n[i])
        arr.append(float(discard(p[i], 4)))
        arr.append(float(discard(temp_v, 4)))
        
        table.loc[i + 1] = arr
        hi_square += temp_v

    display(table)
    print('X_2 = ' + discard(hi_square, 4))

m = 0.0598
sigma =  1.0079



i,delta,n,p_i,temp_v
1,-1,19.0,0.1465,1.2910
2,-0.5,15.0,0.1428,0.0364
3,0,15.0,0.1870,0.7335
4,0.5,13.0,0.1925,2.0299
5,1,21.0,0.1557,1.8941
6,inf,17.0,0.1755,0.0169


X_2 = 6.0019


i,delta,n,p_i,temp_v
1,-1,154.0,0.1465,0.3829
2,-0.857143,41.0,0.0350,1.0438
3,-0.714286,43.0,0.0398,0.2636
4,-0.571429,45.0,0.0443,0.0102
5,-0.428571,48.0,0.0484,0.0039
6,-0.285714,66.0,0.0519,3.8475
7,-0.142857,51.0,0.0545,0.2187
8,0,71.0,0.0560,4.0041
9,0.142857,54.0,0.0565,0.1102
10,0.285714,70.0,0.0558,3.5907


X_2 = 26.3261
